In [32]:
# we will use data of spiral data
def spiral_data(samples, classes):
    X = np.zeros((samples*classes, 2))
    y = np.zeros(samples*classes, dtype='uint8')
    for class_number in range(classes):
        ix = range(samples*class_number, samples*(class_number+1))
        r = np.linspace(0.0, 1, samples)
        t = np.linspace(class_number*4, (class_number+1)*4, samples) + np.random.randn(samples)*0.2
        X[ix] = np.c_[r*np.sin(t*2.5), r*np.cos(t*2.5)]
        y[ix] = class_number
    return X, y

In [33]:
import numpy as np

In [34]:
class Layer_Dense:
# Layer initialization
    def __init__(self, n_inputs, n_neurons):
# Initialize weights and biases
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))
# Forward pass
    def forward(self, inputs):
# Remember input values
        self.inputs = inputs
# Calculate output values from inputs, weights and biases
        self.output = np.dot(inputs, self.weights) + self.biases
# Backward pass
    def backward(self, dvalues):
# Gradients on parameters
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)
# Gradient on values
        self.dinputs = np.dot(dvalues, self.weights.T)

In [35]:
# ReLU activation
class Activation_ReLU:
# Forward pass
    def forward(self, inputs):
        # Remember input values
        self.inputs = inputs
# Calculate output values from inputs
        self.output = np.maximum(0, inputs)
    def backward(self, dvalues):
# Since we need to modify original variable,
# let's make a copy of values first
        self.dinputs = dvalues.copy()
# Zero gradient where input values were negative
        self.dinputs[self.inputs <= 0] = 0

In [36]:
# Softmax activation
class Activation_Softmax:
# Forward pass
    def forward(self, inputs):
# Remember input values
        self.inputs = inputs
# Get unnormalized probabilities
        exp_values = np.exp(inputs - np.max(inputs, axis=1,keepdims=True))
# Normalize them for each sample
        probabilities = exp_values / np.sum(exp_values, axis=1,keepdims=True)
        self.output = probabilities
# Backward pass
    def backward(self, dvalues):
# Create uninitialized array
        self.dinputs = np.empty_like(dvalues)
# Enumerate outputs and gradients
        for index, (single_output, single_dvalues) in enumerate(zip(self.output, dvalues)):
# Flatten output array
            single_output = single_output.reshape(-1, 1)
# Calculate Jacobian matrix of the output and
            jacobian_matrix = np.diagflat(single_output) - np.dot(single_output, single_output.T)
# Calculate sample-wise gradient
# and add it to the array of sample gradients
            self.dinputs[index] = np.dot(jacobian_matrix,single_dvalues)

In [37]:
# Common loss class
class Loss:
# Calculates the data and regularization losses
# given model output and ground truth values
    def calculate(self, output, y):
# Calculate sample losses
        sample_losses = self.forward(output, y)
# Calculate mean loss
        data_loss = np.mean(sample_losses)
# Return loss
        return data_loss

In [38]:
class Loss_CategoricalCrossentropy(Loss):
# Forward pass
    def forward(self, y_pred, y_true):
# Number of samples in a batch
        samples = len(y_pred)
# Clip data to prevent division by 0
# Clip both sides to not drag mean towards any value
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
# Probabilities for target values -
# only if categorical labels
        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[range(samples),y_true]
        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(y_pred_clipped * y_true,axis=1)
# Losses
        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods
# Backward pass
    def backward(self, dvalues, y_true):
# Number of samples
        samples = len(dvalues)
# Number of labels in every sample
# We'll use the first sample to count them
        labels = len(dvalues[0])
# If labels are sparse, turn them into one-hot vector
        if len(y_true.shape) == 1:
            y_true = np.eye(labels)[y_true]
# Calculate gradient
        self.dinputs = -y_true / dvalues
# Normalize gradient
        self.dinputs = self.dinputs / samples


In [39]:
class Activation_Softmax_Loss_CategoricalCrossentropy():
# Creates activation and loss function objects
    def __init__(self):
        self.activation = Activation_Softmax()
        self.loss = Loss_CategoricalCrossentropy()
# Forward pass
    def forward(self, inputs, y_true):
# Output layer's activation function
        self.activation.forward(inputs)
# Set the output
        self.output = self.activation.output
# Calculate and return loss value
        return self.loss.calculate(self.output, y_true)

    def backward(self, dvalues, y_true):
# Number of samples
        samples = len(dvalues)
# If labels are one-hot encoded,
# turn them into discrete values
        if len(y_true.shape) == 2:
            y_true = np.argmax(y_true, axis=1)
# Copy so we can safely modify
        self.dinputs = dvalues.copy()
# Calculate gradient
        self.dinputs[range(samples), y_true] -= 1
# Normalize gradient
        self.dinputs = self.dinputs / samples


In [40]:
# class Optimizer_SGD:
# # Initialize optimizer - set settings,
# # learning rate of 1. is default for this optimizer
#     def __init__(self, learning_rate=1.0):
#         self.learning_rate = learning_rate
# # Update parameters
#     def update_params(self, layer):
#         layer.weights += -self.learning_rate * layer.dweights
#         layer.biases += -self.learning_rate * layer.dbiases

# FOR LEARNING RATE DECAY : -
class Optimizer_SGD:
# Initialize optimizer - set settings,
# learning rate of 1. is default for this optimizer
    def __init__(self, learning_rate=1., decay=0.):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
# Call once before any parameter updates
    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate * (1. / (1. + self.decay * self.iterations))
# Update parameters
    def update_params(self, layer):
        layer.weights += -self.current_learning_rate * layer.dweights
        layer.biases += -self.current_learning_rate * layer.dbiases
# Call once after any parameter updates
    def post_update_params(self):
        self.iterations += 1

In [44]:
# Create dataset
X, y = spiral_data(samples=100, classes=3)
# Create Dense layer with 2 input features and 64 output values
dense1 = Layer_Dense(2, 64)
# Create ReLU activation (to be used with Dense layer):
activation1 = Activation_ReLU()
# Create second Dense layer with 64 input features (as we take output
# of previous layer here) and 3 output values (output values)
dense2 = Layer_Dense(64, 3)
# Create Softmax classifier's combined loss and activation
loss_activation = Activation_Softmax_Loss_CategoricalCrossentropy()
# Create optimizer
optimizer = Optimizer_SGD(learning_rate=1, decay=1e-3)
# Train in loop
for epoch in range(1001):
# Perform a forward pass of our training data through this layer
    dense1.forward(X)
# Perform a forward pass through activation function
# takes the output of first dense layer here
    activation1.forward(dense1.output)
# Perform a forward pass through second Dense layer
# takes outputs of activation function of first layer as inputs
    dense2.forward(activation1.output)
# Perform a forward pass through the activation/loss function
# takes the output of second dense layer here and returns loss
    loss = loss_activation.forward(dense2.output, y)
# Calculate accuracy from output of activation2 and targets
# calculate values along first axis
    predictions = np.argmax(loss_activation.output, axis=1)
    if len(y.shape) == 2:
        y = np.argmax(y, axis=1)
    accuracy = np.mean(predictions==y)
    
    if not epoch % 10:
        print(f'epoch: {epoch}, ' +
              f'acc: {accuracy:.3f}, ' +
              f'loss: {loss:.3f}, ' +
              f'lr: {optimizer.current_learning_rate}')
# Backward pass
    loss_activation.backward(loss_activation.output, y)
    dense2.backward(loss_activation.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)
# Update weights and biases
    optimizer.pre_update_params()
    optimizer.update_params(dense1)
    optimizer.update_params(dense2)
    optimizer.post_update_params()

epoch: 0, acc: 0.390, loss: 1.099, lr: 1
epoch: 10, acc: 0.457, loss: 1.098, lr: 0.9910802775024778
epoch: 20, acc: 0.453, loss: 1.098, lr: 0.9813542688910698
epoch: 30, acc: 0.440, loss: 1.098, lr: 0.9718172983479106
epoch: 40, acc: 0.427, loss: 1.097, lr: 0.9624639076034649
epoch: 50, acc: 0.423, loss: 1.095, lr: 0.9532888465204957
epoch: 60, acc: 0.417, loss: 1.093, lr: 0.9442870632672333
epoch: 70, acc: 0.427, loss: 1.089, lr: 0.9354536950420954
epoch: 80, acc: 0.433, loss: 1.084, lr: 0.9267840593141798
epoch: 90, acc: 0.440, loss: 1.079, lr: 0.9182736455463728
epoch: 100, acc: 0.427, loss: 1.075, lr: 0.9099181073703367
epoch: 110, acc: 0.420, loss: 1.073, lr: 0.9017132551848512
epoch: 120, acc: 0.420, loss: 1.072, lr: 0.8936550491510277
epoch: 130, acc: 0.427, loss: 1.071, lr: 0.8857395925597874
epoch: 140, acc: 0.430, loss: 1.070, lr: 0.8779631255487269
epoch: 150, acc: 0.447, loss: 1.069, lr: 0.8703220191470844
epoch: 160, acc: 0.447, loss: 1.068, lr: 0.8628127696289905
epoch: 1